In [1]:
import requests
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go

firstUserID = 579
# lastUserID = 973
lastUserID = 1142

pd.set_option("display.max_rows", 1000)

In [2]:
# 得到用户统计数据
def getUserStats(uid):
    r = requests.get('https://block7serv.heyalgo.io/v1/games/userstats?token=ewfbneivneghy3uryh&uid={}'.format(uid))
    return r.json()

# 解析时间字符串，2021-10-13_10:01:48
def parseTime(str):
    if str == '':
        return None
    
    return datetime.strptime(str, '%Y-%m-%d_%H:%M:%S')

# 计算时间差，返回整数小时差
def getTimeOffsetHours(timeNow, timeStart):
    if timeStart == None:
        return -1
    
    return int((timeNow.timestamp() - timeStart.timestamp()) / 60 / 60)

# dt = parseTime('2021-10-13_10:01:48')
# datetime.now()
# getTimeOffsetHours(datetime.now(), dt)

In [3]:
# 计算平均关卡星星，需要跳过5的倍数的关卡
def countAvgStageStars(levelArr):
    if levelArr == None:
        return 0
    
    totalstars = 0
    stagenums = 0
    
    for key in levelArr.keys():
        if int(key) % 5 != 0:
            totalstars = totalstars + int(levelArr[key])
            stagenums = stagenums + 1
    
    if stagenums > 0:
        return totalstars / stagenums
    
    return 0
        
# 计算下一关的流失率
def countNextStage(lststages, stage, curnums):
    if curnums > 0:
        for s in lststages:
            if s['stage'] == stage + 1:
                s['lostper'] = (curnums - s['totalusers']) / curnums
                
                if s['lostper'] < 0:
                    s['lostper'] = 0

                return
    else:
        s['lostper'] = 0
        
# 获取前一关的游戏人数       
def getPreStageTotalUsers(lststages, stage):
    for s in lststages:
        if s['stage'] == stage - 1:
            return s['totalusers']
            
    return 0

# new stage stats data
def newStageData(stage, winper):
    ss = {
        'stage': stage,
        'totalwinper': winper,
        'totalusers': 1,
        'winper': winper,
        'totalClickNums': 0,
        'avgClickNums': 0,
        'totalAvgClickTime': 0,
        'avgClickTime': 0,
        'totalnums': 0,
        'totalWinClickNums': 0,
        'avgWinClickNums': 0,
        'totalWinAvgClickTime': 0,
        'avgWinClickTime': 0,
        'totalWinNums': 0,
        'totalLoseClickNums': 0,
        'avgLoseClickNums': 0,
        'totalLoseAvgClickTime': 0,
        'avgLoseClickTime': 0,
        'totalLoseNums': 0,
        'stayUsers': 0,
        'stayUsersPer': 0,    
        'avgLoseProgress': 0,
        'totalGameNums': 0,
        'totalGameWinNums': 0,
        'gameWinPer': 0,        
    }
    
    return ss


# 新增加一个玩家关卡统计
def addUserStages(lststages, stage, winper, winnums, totalnums):
    for s in lststages:
        if s['stage'] == stage:
            s['totalGameWinNums'] = s['totalGameWinNums'] + winnums
            s['totalGameNums'] = s['totalGameNums'] + totalnums
            if s['totalGameNums'] > 0:
                s['gameWinPer'] = s['totalGameWinNums'] / s['totalGameNums']
            
            s['totalwinper'] = s['totalwinper'] + winper
            s['totalusers'] = s['totalusers'] + 1
            s['winper'] = s['totalwinper'] / s['totalusers']
            
            ptnums = getPreStageTotalUsers(lststages, stage)
            if ptnums != 0:
                s['lostper'] = (ptnums - s['totalusers'])  / ptnums
                
                if s['lostper'] < 0:
                    s['lostper'] = 0
            else:
                s['lostper'] = 0
            
            countNextStage(lststages, stage, s['totalusers'])
            
            return
    
    ss = newStageData(stage, winper)
    
    ss['totalGameWinNums'] = ss['totalGameWinNums'] + winnums
    ss['totalGameNums'] = ss['totalGameNums'] + totalnums
    if ss['totalGameNums'] > 0:
        ss['gameWinPer'] = ss['totalGameWinNums'] / ss['totalGameNums']    
    
    lststages.append(ss)
    
    ss['lostper'] = (getPreStageTotalUsers(lststages, stage) - ss['totalusers']) / ss['totalusers']        
    countNextStage(lststages, stage, ss['totalusers'])
    
    
def procUserStayStage(lststages, us):
    for s in lststages:
        if s['stage'] == int(us['user']['level']):
            s['stayUsers'] = s['stayUsers'] + 1
            s['stayUsersPer'] = s['stayUsers'] / s['totalusers']
            
            return


# 分析玩家历史数据
def analyzeUserStageHistory(ui, lststages, stage, stageHistory):
    for s in lststages:
        if s['stage'] == stage:
            s['totalnums'] = s['totalnums'] + 1
            
            s['totalClickNums'] = s['totalClickNums'] + stageHistory['clickNums']
            s['avgClickNums'] = s['totalClickNums'] / s['totalnums'] 
            
            s['totalAvgClickTime'] = s['totalAvgClickTime'] + stageHistory['avgClickTime']
            s['avgClickTime'] = s['totalAvgClickTime'] / s['totalnums']
            
            ui['totalHistoryNums'] = ui['totalHistoryNums'] + 1
            ui['totalAvgClickTime'] = ui['totalAvgClickTime'] + stageHistory['avgClickTime']
            ui['avgClickTime'] = ui['totalAvgClickTime'] / ui['totalHistoryNums']
            
            if stageHistory['gamestate'] == 1:
                s['totalWinNums'] = s['totalWinNums'] + 1

                s['totalWinClickNums'] = s['totalWinClickNums'] + stageHistory['clickNums']
                s['avgWinClickNums'] = s['totalWinClickNums'] / s['totalWinNums'] 

                s['totalWinAvgClickTime'] = s['totalWinAvgClickTime'] + stageHistory['avgClickTime']
                s['avgWinClickTime'] = s['totalWinAvgClickTime'] / s['totalWinNums']                
            else:
                s['totalLoseNums'] = s['totalLoseNums'] + 1

                s['totalLoseClickNums'] = s['totalLoseClickNums'] + stageHistory['clickNums']
                s['avgLoseClickNums'] = s['totalLoseClickNums'] / s['totalLoseNums'] 

                s['totalLoseAvgClickTime'] = s['totalLoseAvgClickTime'] + stageHistory['avgClickTime']
                s['avgLoseClickTime'] = s['totalLoseAvgClickTime'] / s['totalLoseNums']
                
            if s['avgWinClickNums'] > 0:
                s['avgLoseProgress'] = s['avgLoseClickNums'] / s['avgWinClickNums']
                
            return
            
    
# 分析玩家关卡数据    
def analyzeUserStages(ui, lststages, stages):
    if stages == None:
        ui['stagenums'] = 0
        
        return 
    
    nums = 0
    for key in stages.keys():
        if len(stages[key]['historys']) > 0:
            addUserStages(lststages, int(key), stages[key]['winnums'] / len(stages[key]['historys']), 
                          stages[key]['winnums'], len(stages[key]['historys']))
            nums = nums + len(stages[key]['historys'])
            
            for sh in stages[key]['historys']:
                analyzeUserStageHistory(ui, lststages, int(key), sh)
                
            
    ui['stagenums'] = nums
        
    return
    

# 分析用户数据
def analyzeUserStats(startUID, endUID):
    lstui = []
    lststages = []
    timeNow = datetime.now()
    
    for uid in range(startUID, endUID + 1):
        cui = getUserStats(uid)
        ui = {
            'uid': uid, 
            'coin': int(cui['user']['coin']),
            'level': int(cui['user']['level']),
            'createTime': parseTime(cui['user']['createTime']),
            'lastTime': parseTime(cui['user']['lastLoginTime']),
            'avgStars': countAvgStageStars(cui['user']['levelarr']),
            'ip': cui['user']['ip'],
            'totalAvgClickTime': 0,
            'avgClickTime': 0,
            'totalHistoryNums': 0,
        }
        
        if ui['lastTime'] == None:
            ui['lastTime'] = ui['createTime']
        
        ui['offlineHours'] = getTimeOffsetHours(timeNow, ui['lastTime'])
        ui['aliveHours'] = getTimeOffsetHours(ui['lastTime'], ui['createTime'])
        
        analyzeUserStages(ui, lststages, cui['user']['stages'])
        
        procUserStayStage(lststages, cui)
        
        lstui.append(ui)
    
    return lstui, lststages

In [4]:
lstui, lststages = analyzeUserStats(firstUserID, lastUserID)

In [5]:
df = pd.DataFrame(lstui)
df['createTime'] = pd.to_datetime(df['createTime'])

# df[(df['aliveHours'] > 0)&(df['level'] > 0)].sort_values("level")

In [6]:
# df[(df['createTime'] > datetime.strptime('2021-10-13', '%Y-%m-%d'))&(df['createTime'] < datetime.strptime('2021-10-14', '%Y-%m-%d'))]

In [7]:
df1 = pd.DataFrame(lststages)

df1.sort_values("stage")

,stage,totalwinper,totalusers,winper,totalClickNums,avgClickNums,totalAvgClickTime,avgClickTime,totalnums,totalWinClickNums,...,totalLoseAvgClickTime,avgLoseClickTime,totalLoseNums,stayUsers,stayUsersPer,avgLoseProgress,totalGameNums,totalGameWinNums,gameWinPer,lostper
138,1,310.766667,318,0.977254,12125,35.247093,5318.496119,15.460745,344,11806,...,214.974904,13.435931,16,24,0.079734,0.553913,344,328,0.953488,0.000000
141,2,291.433333,303,0.961826,14569,43.489552,5359.596110,15.998794,335,13767,...,354.246615,12.215401,29,20,0.066225,0.614693,335,306,0.913433,0.047170
139,3,266.620635,291,0.916222,15259,43.597143,7669.217495,21.912050,350,13820,...,542.362432,8.747781,62,37,0.128920,0.483675,350,288,0.822857,0.039604
140,4,238.991667,258,0.926324,19299,55.140000,8410.905514,24.031159,350,16379,...,1333.383998,14.815378,90,23,0.089147,0.515023,350,260,0.742857,0.113402
30,5,219.709524,241,0.911658,26263,79.826748,11728.045355,35.647554,329,22762,...,1401.181125,18.436594,76,36,0.151261,0.512022,329,253,0.768997,0.065891
142,6,199.944444,212,0.943134,14390,58.734694,10371.614163,42.333119,245,13158,...,759.697089,23.740534,32,12,0.057416,0.623233,245,213,0.869388,0.120332
143,7,189.416667,197,0.961506,12890,60.516432,6316.331095,29.654137,213,12410,...,327.416838,20.463552,16,15,0.077320,0.476229,213,197,0.924883,0.070755
137,8,173.311508,189,0.916992,17636,67.057034,9450.845030,35.934772,263,15123,...,1276.085662,18.493995,69,54,0.293478,0.467205,263,194,0.737643,0.040609
31,9,133.500000,144,0.927083,15049,78.380208,8320.089525,43.333800,192,12479,...,1124.599697,24.991104,45,18,0.135338,0.672757,192,147,0.765625,0.238095
24,10,127.000000,132,0.962121,13070,87.718121,4729.377690,31.740790,149,12508,...,115.349472,11.534947,10,13,0.100000,0.624544,149,139,0.932886,0.083333


In [9]:
def showStagesStats(df):
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['totalusers'],
        name='参与用户数',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['winper'],
        name='平均用户通关率',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['totalnums'],
        name='挑战次数',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['avgClickTime'],
        name='平均点击时间差',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['avgWinClickTime'],
        name='平均胜利点击时间差',
    ))    
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['avgLoseClickTime'],
        name='平均失败点击时间差',
    ))    
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['lostper'],
        name='流失率',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['stayUsers'],
        name='滞留用户数',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['stayUsersPer'],
        name='滞留用户比例',
    ))    
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['avgLoseProgress'],
        name='平均失败进度',
    ))
    fig.add_trace(go.Bar(
        x=df['stage'],
        y=df['gameWinPer'],
        name='通关率',
    ))    

    fig.update_layout(barmode='group', xaxis_tickangle=-45)
    fig.show()
    
showStagesStats(df1)